formatting tips: https://gist.github.com/nikhilnayyar002/7a35e653d3d590e317c829243e73b110
or chatgpt idk

By: Andrew Yang, Megan Yu, and Frank Deegbe

# Part 1: Data Curation
### NFL-Data
Data extracted from Fantasy.NFL.com divided by season and player position, along with weekly numbers for the 2021 season.

The extract goes back to the 2015 season as that is the first available season with a full dataset for all positions.

Point total and player-ranks are specific for the MML leagues point system but all outer data are raw and can be used for other calculations and simulations.

All data are available in comma-separated csv files or row-divided JSON.

During this project, we will be using Python language, and we use tools such as [VS Code](https://code.visualstudio.com/) and [Jupiter Notebook](https://jupyter.org/) to develop this project.

Just like any other Python project, we need to import some libraries. Here are some of the libraries we will be using throughout this tutorial.

<center><b>Imports</b></center>


In [4]:
# !pip install numpy pandas seaborn matplotlib folium
import os
import folium
import warnings
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# this is just an example idk if we gonna need em

From NFL-Data git repo, we get data for running backs for years 2020-2023. We read each year into a separate dataframe. Displayed below are the top five running backs of 2023, as well as their statistics. Their positive contributions are input into a formula which outputs their "Total Points" - the fianl column. We will use Total Points to rank players to analyze their performance.

In [10]:
years = ["2020", "2021", "2022", "2023"]
# array of dataframes containing RB data for years 2020-2023
rb_dfs = []
for year in years:
    rb_url = "https://raw.githubusercontent.com/hvpkod/NFL-Data/main/NFL-data-Players/"+ year +"/RB_season.csv"
    rb_dfs.append(pd.read_csv(rb_url))

rb_dfs[-1].head()

,PlayerName,PlayerId,Pos,Team,PassingYDS,PassingTD,PassingInt,RushingYDS,RushingTD,ReceivingRec,...,TouchReceptions,Touches,TargetsReceptions,Targets,ReceptionPercentage,RzTarget,RzTouch,RzG2G,Rank,TotalPoints
0,Christian McCaffrey,2557997,RB,SF,NaN,NaN,NaN,1459.0,14.0,67.0,...,67.0,339.0,67.0,83.0,81.0,16.0,75.0,39.0,1,359.80
1,Travis Etienne,2566157,RB,JAX,NaN,NaN,NaN,1008.0,11.0,58.0,...,58.0,325.0,58.0,73.0,79.0,2.0,39.0,15.0,2,256.40
2,Breece Hall,2568030,RB,NYJ,NaN,NaN,NaN,994.0,5.0,76.0,...,76.0,299.0,76.0,95.0,80.0,5.0,25.0,17.0,3,254.50
3,Raheem Mostert,2553728,RB,MIA,NaN,NaN,NaN,1012.0,18.0,25.0,...,25.0,234.0,25.0,32.0,78.0,5.0,50.0,32.0,4,250.17
4,Joe Mixon,2557976,RB,CIN,NaN,NaN,NaN,1034.0,9.0,52.0,...,52.0,309.0,52.0,64.0,81.0,13.0,67.0,42.0,5,241.00
